# Import

In [1]:
import csv
import os
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_mutual_info_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from smart_open import open

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
tqdm.pandas()

# Read data

In [3]:
newsgroups_df = pd.read_csv("../../../Preprocessing/data/20News/master.csv", index_col=0)

In [4]:
with open("../../../Preprocessing/data/20News/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [5]:
vector_sizes = [2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [6]:
class Corpus:
    def __init__(self, path=None, texts=None):
        self.path = path
        self.texts = texts
        self.dictionary = Dictionary(texts)

    def __iter__(self):
        if self.path is not None:
            for line in open(self.path):
                # assume there's one document per line, tokens separated by whitespace
                yield self.dictionary.doc2bow(line.lower().split())
        else:
            for line in self.texts:
                yield self.dictionary.doc2bow(line)

In [7]:
texts = newsgroups_df.words_nonstop.progress_apply(lambda x: x.split(' ')).tolist()
corpus = Corpus(texts=texts)
dictionary = Dictionary(texts)

100%|██████████| 18846/18846 [00:01<00:00, 14546.11it/s]


In [8]:
label = newsgroups_df["class"].to_numpy()

In [9]:
model_nums = 2
vector_model_num = 0

In [10]:
vectors_path = "../../../Clustering/data/20News/doc2vec/vector/"
models_path = "../../../Clustering/data/20News/doc2vec/GMM/model/"
pred_path = "../../../Clustering/data/20News/doc2vec/GMM/pred/"

coherences = []
for model_num in range(model_nums):
    coherence = []
    for vector_size in tqdm(vector_sizes):
        vector_path = f"{vectors_path}{vector_size}/normalized/{vector_model_num}.csv"
        vectors = np.loadtxt(vector_path, delimiter=",")

        gmm = pickle.load(open(f"{models_path}{vector_size}/{model_num}.sav", "rb"))
        pred = gmm.predict(vectors)

        cm = CoherenceModel(
            topics=pred.reshape(1, -1),
            corpus=corpus,
            dictionary=dictionary,
            texts=texts,
            coherence="c_v",
        )
        coherence.append(cm.get_coherence())
    coherences.append(coherence)
    

100%|██████████| 14/14 [19:49<00:00, 84.97s/it] 


In [11]:
pd.DataFrame(coherences)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.516721,0.553618,0.501803,0.508242,0.524243,0.514786,0.537837,0.513836,0.502633,0.529951,0.502572,0.520165,0.560618,0.512547
1,0.522438,0.567989,0.535833,0.542723,0.517592,0.550262,0.510531,0.513212,0.505539,0.540728,0.538972,0.559448,0.524159,0.525926
